In [ ]:
# -converting pdf to text
# -text to kg(emtities, relationship, embeddings)
# -retrival()

In [38]:
# #!pip install --upgrade pymilvus
# !pip install git+https://github.com/zc277584121/graphrag.git

In [37]:
api_key = '9101287fb67e484b9970dd8d7e31aa05' # Your OpenAI API key
llm_model = "gpt-4o"  # Or gpt-4-turbo-preview
embedding_model = "text-embedding-ada-002"

In [38]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [137]:
import os  
  
def get_latest_timestamp_dir(base_dir):  
    # List all directories in the base directory  
    all_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]  
      
    # Sort the directories based on their names assuming they are timestamped  
    all_dirs.sort(reverse=True)  
      
    # Return the latest directory  
    return all_dirs[0] if all_dirs else None  
  
# Example usage  
OUTPUT_DIR = r"output"  
latest_timestamp = get_latest_timestamp_dir(OUTPUT_DIR)  
  
if latest_timestamp:  
    INPUT_DIR = os.path.join(OUTPUT_DIR, latest_timestamp, "artifacts")  
    print(INPUT_DIR)  
else:  
    print("No directories found.")  


output\20241019-145519\artifacts


In [138]:
INPUT_DIR = INPUT_DIR
# diesl-graph\classical_graphrag\output\20241019-143614
# diesl-graph\classical_graphrag\output\20241019-120138
# diesl-graph\classical_graphrag\output\20241019-120138
# diesl-graph\classical_graphrag\output\20241019-114417
# diesl-graph\classical_graphrag\output\20241014-023025
# diesl-graph\classical_graphrag\output\20241014-022450
# diesl-graph\classical_graphrag\output\20241014-021151
# diesl-graph\classical_graphrag\output\20241014-011736
LANCEDB_URI = f"{INPUT_DIR}/lancedb"
# diesl-graph\classical_graphrag\output\20241013-131846
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [139]:
# import os  
  
# # Get the current working directory  
# current_path = os.getcwd()  
# print("Current Path:", current_path)  


In [140]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 207


,level,title,type,description,source_id,degree,human_readable_id,id,size,graph_embedding,community,top_level_node_id,x,y
0,0,DEW CONSTRUCTION,"COMPANY, ORGANIZATION",A construction company located at 277 Blair Pa...,57fe050b01845cd18779ab7650252c67,4,0,b45241d70f0e43fca764df95b2b81f77,4,None,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,CHARLOTTE CENTRAL SCHOOL,"PROJECT, LOCATION","A school located at 408 Hinesburg Road, Charlo...",57fe050b01845cd18779ab7650252c67,2,1,4119fd06010c494caa07f439b333f4c5,2,None,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,GYPSUM WALL BOARD,"MATERIAL, PRODUCT",A construction material used in wall and ceili...,57fe050b01845cd18779ab7650252c67,1,2,d3835bf3dda84ead99deadbeac5d0d7d,1,None,4,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,BECKY ST. GEORGE,"PERSON, ROLE","Submittal Manager at DEW Construction, respons...",57fe050b01845cd18779ab7650252c67,1,3,077d2820ae1845bcbb1803379a3d1eae,1,None,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,STEEL ELEMENTS INTERNATIONAL,"COMPANY, CONTRACTOR","The contractor responsible for the submittal, ...",57fe050b01845cd18779ab7650252c67,1,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,1,None,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


In [125]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 83


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,DEW CONSTRUCTION,CHARLOTTE CENTRAL SCHOOL,1.0,DEW Construction is managing the renovation pr...,[57fe050b01845cd18779ab7650252c67],babe97e1d9784cffa1c85abc1e588126,0,4,2,6
1,DEW CONSTRUCTION,BECKY ST. GEORGE,1.0,Becky St. George is the Submittal Manager at D...,[57fe050b01845cd18779ab7650252c67],1033a18c45aa4584b2aef6ab96890351,1,4,1,5
2,DEW CONSTRUCTION,MIKE WANDERLICH,1.0,Mike Wanderlich is involved in the submittal r...,[57fe050b01845cd18779ab7650252c67],c9b8ce91fc2945b4907fe35519339cac,2,4,1,5
3,DEW CONSTRUCTION,LUKE KEENAN,1.0,Luke Keenan is a reviewer at DEW Construction,[57fe050b01845cd18779ab7650252c67],fa3c4204421c48609e52c8de2da4c654,3,4,1,5
4,CHARLOTTE CENTRAL SCHOOL,STEEL ELEMENTS INTERNATIONAL,1.0,Steel Elements International is the contractor...,[57fe050b01845cd18779ab7650252c67],53af055f068244d0ac861b2e89376495,4,2,1,3


In [126]:
# # NOTE: covariates are turned off by default, because they generally need prompt tuning to be valuable
# # Please see the GRAPHRAG_CLAIM_* settings
# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

# claims = read_indexer_covariates(covariate_df)

# print(f"Claim records: {len(claims)}")
# covariates = {"claims": claims}

In [127]:
api_key = '9101287fb67e484b9970dd8d7e31aa05' # Your OpenAI API key
llm_model = "gpt-4o"  # Or gpt-4-turbo-preview
embedding_model = "text-embedding-ada-002"

In [128]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 16


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,14,# Gypsum Board and Associated Standards\n\nThe...,2,7.5,Gypsum Board and Associated Standards,The impact severity rating is high due to the ...,"The community centers around Gypsum Board, a k...",[{'explanation': 'Gypsum Board is a versatile ...,"{\n ""title"": ""Gypsum Board and Associated S...",175cefb9-42b1-431d-b9b3-016db7671eac
1,15,# Gold Bond Building Products and National Gyp...,2,6.5,Gold Bond Building Products and National Gypsu...,The impact severity rating is moderate to high...,The community is centered around Gold Bond Bui...,"[{'explanation': 'Gold Bond Building Products,...","{\n ""title"": ""Gold Bond Building Products a...",a7f5cd2f-be33-4a62-aa50-b18df76fc2eb
2,10,# Georgia-Pacific Gypsum and ToughRock Firegua...,1,7.5,Georgia-Pacific Gypsum and ToughRock Fireguard...,The impact severity rating is high due to the ...,The community centers around Georgia-Pacific G...,[{'explanation': 'Georgia-Pacific Gypsum is a ...,"{\n ""title"": ""Georgia-Pacific Gypsum and To...",f212b414-d7a2-4df0-9654-a1b287186069
3,11,# Georgia-Pacific Gypsum LLC and ToughRock Fir...,1,6.5,Georgia-Pacific Gypsum LLC and ToughRock Fireg...,The impact severity rating is moderate due to ...,The community centers around Georgia-Pacific G...,[{'explanation': 'Georgia-Pacific Gypsum LLC i...,"{\n ""title"": ""Georgia-Pacific Gypsum LLC an...",5a074a44-1122-418f-a77c-2ac15b375b19
4,12,# GRIDMARX and XP Hi-Abuse Gypsum Board Commun...,1,7.5,GRIDMARX and XP Hi-Abuse Gypsum Board Community,The impact severity rating is high due to the ...,This community centers around GRIDMARX and XP ...,[{'explanation': 'GRIDMARX is a preprinted fas...,"{\n ""title"": ""GRIDMARX and XP Hi-Abuse Gyps...",2406900d-ad99-4533-8c0d-32d595cf074f


In [129]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 11


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,57fe050b01845cd18779ab7650252c67,---- Page 1 ----\nDEW\nCONSTRUCTION\nDEW Const...,1500,[ca5cbf4db4c8e3ba35a4547210722d5a],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[babe97e1d9784cffa1c85abc1e588126, 1033a18c45a..."
1,488f6c425b2a94e2ec6634bd28490f37,Advantages\n· Specially designed gypsum core w...,1500,[ca5cbf4db4c8e3ba35a4547210722d5a],"[1fd3fa8bb5a2408790042ab9573779ee, 27f9fbe6ad8...","[fc01e9baa80e417c9206f941bb279407, 56d0e5ebe79..."
2,490a4f510b4c01e1699c0cd209b10744,"� 5/8"" (16 mm)\nNail Pull Resistance1\n≥ 87 lb...",1500,[ca5cbf4db4c8e3ba35a4547210722d5a],"[1fd3fa8bb5a2408790042ab9573779ee, 27f9fbe6ad8...","[fc01e9baa80e417c9206f941bb279407, 0adb2d9941f..."
3,08457fe8eb8bcba5634aa44b157d25c2,up on the field\nnails. The total quantity of...,1500,[ca5cbf4db4c8e3ba35a4547210722d5a],"[1fd3fa8bb5a2408790042ab9573779ee, 27f9fbe6ad8...","[fc01e9baa80e417c9206f941bb279407, 0adb2d9941f..."
4,6eb55ef89dc98e83dabfce0347f7ae69,a vapor\nretarder in exterior ceilings behind...,1500,[ca5cbf4db4c8e3ba35a4547210722d5a],"[1fd3fa8bb5a2408790042ab9573779ee, 27f9fbe6ad8...","[fc01e9baa80e417c9206f941bb279407, 7c49f2710e8..."


In [130]:
api_key = "9101287fb67e484b9970dd8d7e31aa05"
# llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
# embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
api_base="https://diesl-eus-openai-dev.openai.azure.com/"
api_version="2024-02-15-preview"
api_key = '9101287fb67e484b9970dd8d7e31aa05' # Your OpenAI API key
llm_model = "gpt-4o"  # Or gpt-4-turbo-preview
embedding_model = "text-embedding-ada-002"
llm = ChatOpenAI(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=4,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    api_type=OpenaiApiType.AzureOpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=3,
)

In [131]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None

    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [132]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [133]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [134]:
result = await search_engine.asearch("""
Step 1: Leverage the 'Section Includes' portion of the document to identify the key product(s) within the text document.
Step 2: Identify the aliases used to refer to the key products outlined within the 'Section Includes' portion of the PDF.
	a) Example aliases include 'GYPBD-1, RF-1'. Return only a numbered list of products that have an alias.
Step 3: Using the list of products and aliases you created in step 2, verify  no product or alias is excluded from our list. If you identify a new product and alias, add this new product to the existing list. 


""")
print(result.response)

I'm sorry, but I don't have access to the specific "Section Includes" portion of the document you're referring to. If you can provide that section or more details, I can help analyze the information based on the data available.


In [29]:
question = """
what is the document about  
"""
result = await search_engine.asearch(question)
print(result.response)

CancelledError: 

In [78]:
!python -m graphrag.query --root . --method local """

Step 1: Leverage the 'Section Includes' portion of the document to identify the key product(s) within the PDF.  
Step 2: Identify the aliases used to refer to the key products outlined within the 'Section Includes' portion of the PDF.  
  a) Example aliases include 'GYPBD-1, RF-1'  
  Return only a numbered list of products that have an alias.  
Step 3: Using the list of products and aliases you created in step 2, verify no product or alias is excluded from our list. If you identify a new product and alias, add this new product to the existing list.  
"""


SyntaxError: unmatched ')' (2098973318.py, line 5)

In [ ]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

c:\Users\aziz1\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
